In [1]:
# Import the usual libraries
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
#import matplotlib.patches as mpatches

# Enable inline plotting
%matplotlib inline

#from IPython.display import display, Latex, clear_output

In [2]:
import pynrc
from pynrc import nrc_utils
from pynrc.nrc_utils import webbpsf, poppy

[     pynrc:INFO]   jwst_backgrounds is not installed and will not be used for bg estimates.


In [3]:
from pynrc.nrc_utils import read_filter, gen_psf_coeff, gen_image_coeff, gen_webbpsf_psf
from pynrc.nrc_utils import gen_webbpsf_siwfe, field_coeff_resid, field_coeff_func
from pynrc.nrc_utils import Tel2Sci_info, NIRCam_V2V3_limits, radial_std
from pynrc.nrc_utils import jl_poly_fit, jl_poly

In [4]:
# Interpolation and extrapolation
from scipy.interpolate import griddata, RegularGridInterpolator
from scipy.ndimage import rotate

In [5]:
# pySIAF stuff for plotting
import pysiaf
from pysiaf.siaf import Siaf
from pysiaf.siaf import plot_main_apertures

siaf = Siaf('NIRCam')
siaf.generate_toc()

In [6]:
from astropy.table import Table

# Benchmarks

In [7]:

def perform_benchmarks(filter='F430M', pupil=None, mask=None, module='A',
                       fov_pix=33, oversample=4, include_si_wfe=True, 
                       use_legendre=True, force=False, save=True, 
                       do_webbpsf=True, do_webbpsf_only=False, return_nrc=False,
                       use_mp=None, nproc=None, **kwargs):
    
    import datetime, time
    
    # PSF setup
#     kwargs = {}
    kwargs['pupil'] = 'CLEAR' if pupil is None else pupil
    kwargs['mask'] = mask
    kwargs['module'] = module

    kwargs['fov_pix'] = fov_pix
    kwargs['oversample'] = oversample

    kwargs['opd'] = ('OPD_RevW_ote_for_NIRCam_requirements.fits.gz', 0)
    kwargs['jitter'] = 'gaussian'
    kwargs['jitter_sigma'] = 0.007

    kwargs['include_si_wfe'] = include_si_wfe
    kwargs['use_legendre'] = use_legendre

    kwargs['force']     = force
    kwargs['save']      = save
    kwargs['save_name'] = None
    
    pynrc.setup_logging('WARN', verbose=False)
    
    tdict = {
        'webbpsf_init': None,
        'webbpsf_psf': None,
        'pynrc_coeff': None,
        'pynrc_drift': None,
        'pynrc_field': None,
        'pynrc_psf': None,
    }
    
    # Multiprocessing cases
    use_mp_def = poppy.conf.use_multiprocessing
    nproc_def = poppy.conf.n_processes
    if use_mp is not None:
        poppy.conf.use_multiprocessing = use_mp
    if nproc is not None:
        poppy.conf.n_processes = nproc

    ####
    # WebbPSF timings
    ####
    if do_webbpsf or do_webbpsf_only:
        # Initialization overheads
        tarr = []
        for i in range(5):
            t0 = time.time()
            inst = webbpsf.NIRCam()
            inst.detector = 'NRCA5'
            t1 = time.time()
            tarr.append(t1-t0)
        tdict['webbpsf_init'] = dt = np.mean(tarr)
        time_string = 'Took {:.2f} seconds to init WebbPSF'.format(dt)
        print(time_string)    

        # PSF Geneation
        bp = read_filter(filter, **kwargs)
        tarr = []
        for i in range(5):
            t0 = time.time()
            hdul = gen_webbpsf_psf(bp, wfe_drift=5, **kwargs)
            t1 = time.time()
            dt = t1-t0
            tarr.append(dt)
            if dt>10:
                break
        tdict['webbpsf_psf'] = dt = np.mean(tarr) - tdict['webbpsf_init'] 
        time_string = 'Took {:.2f} seconds to generate WebbPSF PSF'.format(dt)
        print(time_string)    

    if do_webbpsf_only and (not return_nrc):
        return tdict
    
    ####
    # pyNRC coefficients
    ####
    t0 = time.time()
    nrc = pynrc.NIRCam(filter=filter, **kwargs)
    t1 = time.time()
    
    if do_webbpsf_only and return_nrc:
        return nrc

    tdict['pynrc_coeff'] = dt = t1-t0
    time_string = 'Took {:.2f} seconds to generate pyNRC coefficients'.format(dt)
    print(time_string)

    ####
    # WFE drift coefficient generation
    ####
    t0 = time.time()
    nrc.wfe_drift = True
    t1 = time.time()
    dt = t1-t0

    tdict['pynrc_drift'] = dt = t1-t0
    time_string = 'Took {:.2f} seconds to generate WFE Drift coefficients'.format(dt)
    print(time_string)

    ####
    # Field coefficient generation
    ####
    t0 = time.time()
    nrc.wfe_field = True
    t1 = time.time()

    tdict['pynrc_field'] = dt = t1-t0
    time_string = 'Took {:.2f} seconds to generate WFE Field coefficients'.format(dt)
    print(time_string)

    # Add aperture manually to save time
    # This doesn't really matter, just needs to be something plausible
    apname = nrc.get_siaf_apname()
    nrc._siaf_ap = nrc.siaf_nrc[apname]
    
    tarr = []
    for i in range(10):
        t0 = time.time()
        psf0, psf0_over = nrc.gen_psf(wfe_drift=5, coord_vals=(1024,1024), coord_frame='sci', 
                                      return_oversample=True)
        t1 = time.time()
        dt = t1-t0
        tarr.append(dt)
        if dt>10:
            break
    tdict['pynrc_psf'] = dt = np.mean(tarr)
    time_string = 'Took {:.2f} seconds to generate pynrc PSF'.format(dt)
    print(time_string)
    
    # Return defaults
    poppy.conf.use_multiprocessing = use_mp_def
    poppy.conf.n_processes = nproc_def
    
    if return_nrc:
        return nrc
    else:
        return tdict


In [16]:
kwargs = {
    'fov_pix': 257,
    'oversample': 4,
#     'pupil': 'CIRCLYOT',
}

In [14]:
tdict = perform_benchmarks(do_webbpsf_only=True, **kwargs)

Took 2.65 seconds to init WebbPSF
Took 3.18 seconds to generate WebbPSF PSF


In [15]:
tdict = perform_benchmarks(do_webbpsf=False, **kwargs)

Took 4.46 seconds to generate pyNRC coefficients
Took 7.65 seconds to generate WFE Drift coefficients


[     pynrc:WARNING] Generating field-dependent coefficients. This may take some time...


V2/V3 Coordinates and det pixel (sci) on NRCA5/NRCA5_FULL: (0.87, -8.78), (1561.1, 501.2)
V2/V3 Coordinates and det pixel (sci) on NRCA5/NRCA5_FULL: (0.45, -8.39), (1957.9, 865.5)
V2/V3 Coordinates and det pixel (sci) on NRCA5/NRCA5_FULL: (2.40, -8.08), (95.6, 1157.3)
V2/V3 Coordinates and det pixel (sci) on NRCA5/NRCA5_FULL: (1.60, -9.22), (872.6, 83.1)
V2/V3 Coordinates and det pixel (sci) on NRCA5/NRCA5_FULL: (1.28, -7.26), (1176.0, 1941.6)
V2/V3 Coordinates and det pixel (sci) on NRCA5/NRCA5_FULL: (0.88, -7.68), (1558.1, 1541.3)
V2/V3 Coordinates and det pixel (sci) on NRCA5/NRCA5_FULL: (2.00, -8.79), (488.4, 482.3)
V2/V3 Coordinates and det pixel (sci) on NRCA5/NRCA5_FULL: (1.99, -7.68), (494.2, 1542.1)
V2/V3 Coordinates and det pixel (sci) on NRCA5/NRCA5_FULL: (1.16, -8.60), (1287.8, 664.0)
V2/V3 Coordinates and det pixel (sci) on NRCA5/NRCA5_FULL: (0.44, -9.22), (1958.2, 82.1)
V2/V3 Coordinates and det pixel (sci) on NRCA5/NRCA5_FULL: (2.43, -9.21), (89.3, 82.8)
V2/V3 Coordinate

[     pynrc:WARNING] Interpolating coefficient residuals onto regular grid...
[     pynrc:WARNING] Done.


Took 325.69 seconds to generate WFE Field coefficients
Took 0.92 seconds to generate pynrc PSF


In [17]:
tdict = perform_benchmarks(do_webbpsf=False, **kwargs)

Took 4.94 seconds to generate pyNRC coefficients
Took 10.73 seconds to generate WFE Drift coefficients
Took 13.87 seconds to generate WFE Field coefficients
Took 1.06 seconds to generate pynrc PSF


In [24]:
tdict = perform_benchmarks(fov_pix=33, force=False, do_webbpsf=True, nproc=1)

Took 23.42 seconds to generate WebbPSF PSF
Took 2.73 seconds to generate pyNRC coefficients
Took 4.38 seconds to generate WFE Drift coefficients
Took 6.57 seconds to generate WFE Field coefficients
Took 0.06 seconds to generate pynrc PSF


In [23]:
tdict = perform_benchmarks(fov_pix=33, force=False, do_webbpsf=True)

Took 17.85 seconds to generate WebbPSF PSF
Took 2.85 seconds to generate pyNRC coefficients
Took 4.63 seconds to generate WFE Drift coefficients
Took 7.94 seconds to generate WFE Field coefficients
Took 0.11 seconds to generate pynrc PSF


{'webbpsf': 17.852298974990845,
 'pynrc_coeff': 2.846261739730835,
 'pynrc_drift': 4.6318581104278564,
 'pynrc_field': 7.942458152770996,
 'pynrc_psf': 0.10760819911956787}

In [14]:
res.siaf_ap

'NRCA5_FULL'

In [14]:
nrc = webbpsf.NIRCam()
%time psf = nrc.calc_psf(fov_pixels=33, oversample=4) 

CPU times: user 3.1 s, sys: 1.63 s, total: 4.73 s
Wall time: 15 s


In [15]:
webbpsf.webbpsf_core.poppy.conf.n_processes

3

In [17]:
pynrc.nrc_utils.

<function pynrc.nrc_utils.nproc_use(fov_pix, oversample, nwavelengths, coron=False)>

In [16]:
webbpsf.webbpsf_core.poppy.conf.use_multiprocessing

True

In [15]:
res.siaf_nrc['NRCA5_FULL']

<pysiaf.Aperture object AperName=NRCA5_FULL >